In [1]:
from xgboost import XGBClassifier
import xgboost as xgb

import pandas as pd
import numpy as np 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import log_loss

from matplotlib import pyplot
import seaborn as sns
%matplotlib inline


In [8]:
y_train = pd.read_csv('./meinian_round1_train_20180408.csv',encoding="gb2312")
y_train.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,002d1e4859fafd9ded2a2e1e7c839b62,165,100,2.08,1.29,3.24
1,92dd479df5e30ab6a0a1cf85ac53efc3,141,97,2.64,1.36,4.75
2,6bb59d517c4c70f8f50844d24fbd0355,120,80,1.37,1.25,2.66
3,0ebb42adae512906f7e1135da734ea63,100,70,1.27,2.21,1.73
4,ebe7811e919109c42c092abbd98b4ca6,110,80,0.8,1.87,2.21


In [12]:
x_train1=pd.read_table('./meinian_round1_data_part1_20180408.txt',sep='$')
x_train1.head()

,vid,table_id,field_results
0,002d1e4859fafd9ded2a2e1e7c839b62,2403,72.9
1,002d1e4859fafd9ded2a2e1e7c839b62,2404,166.5
2,002d1e4859fafd9ded2a2e1e7c839b62,2405,26.30
3,002d1e4859fafd9ded2a2e1e7c839b62,2420,79
4,002d1e4859fafd9ded2a2e1e7c839b62,0702,无


In [13]:
x_train2=pd.read_table('./meinian_round1_data_part2_20180408.txt',sep='$')
x_train2.head()


,vid,table_id,field_results
0,002d1e4859fafd9ded2a2e1e7c839b62,100005,12.9
1,002d1e4859fafd9ded2a2e1e7c839b62,100007,0.26
2,002d1e4859fafd9ded2a2e1e7c839b62,279006,35.0
3,002d1e4859fafd9ded2a2e1e7c839b62,31,4.82
4,002d1e4859fafd9ded2a2e1e7c839b62,312,6.3


In [14]:
test = pd.read_csv('./meinian_round1_test_a_20180408.csv',encoding="gb2312")
test.head()


,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,c643a744e2e94f3ff354d920958bd37b,NaN,NaN,NaN,NaN,NaN
1,a81c1fc0cfa8c0fbc450d2ffeae14b9f,NaN,NaN,NaN,NaN,NaN
2,2af0da8f6f0b8d642f88b3a8194408b6,NaN,NaN,NaN,NaN,NaN
3,2bd361a4c5b9b0d5aab62c101daa4ee6,NaN,NaN,NaN,NaN,NaN
4,16107e514ed0f51c4e39b1a0fa5a04f9,NaN,NaN,NaN,NaN,NaN


In [25]:
# 组合训练特征
# vid为客户id, table_id为检查项目， fields_results为当前检查项目的检查结果
train = pd.concat([x_train1, x_train2])
train.columns
train_featur_columns = train['table_id'].unique()
train_df = pd.DataFrame(data=None, columns=train_featur_columns)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 2795 entries, 2403 to 2478
dtypes: object(2795)
memory usage: 0.0+ bytes


In [28]:
train_df.head()

,2403,2404,2405,2420,0702,0703,0705,0706,0707,0709,...,A69002,A69003,A69004,A69006,A69007,200001,2283,21A020,2237,2478


In [35]:
len(train['vid'].unique())


57298

In [37]:
y_train.shape

(38199, 6)

In [41]:
# 看起来 收集到特征的人和有检测结果的人数对不上
train = train.drop_duplicates(['vid', 'table_id'], keep='first', inplace=False)
pivoted = train.pivot('vid', 'table_id', 'field_results')

In [42]:
pivoted.head()

table_id,004997,0101,0102,0104,0105,0106,0107,0108,0109,0111,...,Y19001,Y19004,Y29001,Y29002,Y29003,Y29004,Y29005,Y29021,Y29058,Y79001
vid,,,,,,,,,,,,,,,,,,,,,
000330ad1f424114719b7525f400660b,None,双侧甲状腺大小形态正常，包膜光整，实质回声均匀，光点稍粗，未见明显异常回声。CDFI：血流显...,甲状腺彩超（含颈部淋巴细胞）未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
000381f0069cbf7537e6aac8923034ae,None,None,左肾、右肾、膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0003848ebd8d8163603760d53d975693,None,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，未见异常血流。",左肾、右肾、膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
000401cbf304d5a8bd862a81bacfa494,None,"双侧乳腺腺体层轻度增厚,内部结构稍紊乱,回声欠均匀，CDFI：未见异常血流。",肝:脂肪肝（轻度）胆、胰、脾、左肾、右肾未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
00044a586c249c05f0969e45ef03ab9d,None,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，右侧内可见一低回声区，范围约21×15mm，边界...","双侧乳腺小叶增生右乳腺低回声区，考虑结节,建议医院进一步检查",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [44]:
pivoted.shape


(57298, 2795)

In [48]:
# 通过vid merge 各个特征和目标result的5项指标
pivoted.columns


Index(['004997', '0101', '0102', '0104', '0105', '0106', '0107', '0108',
       '0109', '0111',
       ...
       'Y19001', 'Y19004', 'Y29001', 'Y29002', 'Y29003', 'Y29004', 'Y29005',
       'Y29021', 'Y29058', 'Y79001'],
      dtype='object', name='table_id', length=2795)

In [50]:
pivoted.index


Index(['000330ad1f424114719b7525f400660b', '000381f0069cbf7537e6aac8923034ae',
       '0003848ebd8d8163603760d53d975693', '000401cbf304d5a8bd862a81bacfa494',
       '00044a586c249c05f0969e45ef03ab9d', '0004600e422e59d3a3af6ca2af1ff29f',
       '00047fa678726f7847c637705cb0da24', '000650b0e8041d01ab909f9be3cb31c3',
       '0007273e4495aa6bba7c0767a9b28b36', '000963f8c8445536d704bfa618df8cd0',
       ...
       'fff622a54c7ac9fe739f8131220dff5e', 'fff68a16e9adc2532f985f479797b447',
       'fff7034dd98a87d54aa27c000c4e1c27', 'fff7c67aceccc3fdfeaf8747cca2bf8a',
       'fff8c189a8df2e61e11bb8b5f44b0216', 'fff967b31ae549d813d8bb55ba697889',
       'fff98757eb2436135a88112c7a1e8fa8', 'fffaea94731f7253120410a822eb6a30',
       'fffd3d97d6887ff841a5ee12078076c2', 'fffda084a188b379e403f64e62c2adf5'],
      dtype='object', name='vid', length=57298)

In [56]:
# 注意到训练数据中 特征数据和 检查结果数据条数不一样，部分vid病人没有检查结果，是不是不应该用inner merge 应该用outer merge，先用outer merge
# y_train只有38199条数据
# pivoted 有57298条数据
train = pd.merge(pivoted, y_train, left_index=True, right_on='vid', how='outer')

In [57]:
train.head()


,004997,0101,0102,0104,0105,0106,0107,0108,0109,0111,...,Y29005,Y29021,Y29058,Y79001,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
11753,None,双侧甲状腺大小形态正常，包膜光整，实质回声均匀，光点稍粗，未见明显异常回声。CDFI：血流显...,甲状腺彩超（含颈部淋巴细胞）未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,000330ad1f424114719b7525f400660b,114,84,1.4,1.34,2.29
38198,None,None,左肾、右肾、膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,000381f0069cbf7537e6aac8923034ae,NaN,NaN,NaN,NaN,NaN
9477,None,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，未见异常血流。",左肾、右肾、膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,0003848ebd8d8163603760d53d975693,107,69,1.33,1.64,4.49
31916,None,"双侧乳腺腺体层轻度增厚,内部结构稍紊乱,回声欠均匀，CDFI：未见异常血流。",肝:脂肪肝（轻度）胆、胰、脾、左肾、右肾未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,000401cbf304d5a8bd862a81bacfa494,143,93,0.73,1.29,1.66
4944,None,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，右侧内可见一低回声区，范围约21×15mm，边界...","双侧乳腺小叶增生右乳腺低回声区，考虑结节,建议医院进一步检查",None,None,None,None,None,None,None,...,None,None,None,None,00044a586c249c05f0969e45ef03ab9d,99,71,1.63,1.51,2.68


In [58]:
train.shape


(57298, 2801)

In [59]:
test.shape

(9550, 6)

In [60]:
# 看起来刚刚那部分 缺失检查结果的特征 是属于test的
test = pd.merge(pivoted, test, left_index=True, right_on='vid', how='inner')

In [61]:
test.head()


,004997,0101,0102,0104,0105,0106,0107,0108,0109,0111,...,Y29005,Y29021,Y29058,Y79001,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
7472,None,None,左肾、右肾、膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,000381f0069cbf7537e6aac8923034ae,NaN,NaN,NaN,NaN,NaN
2253,None,"双侧颈总动脉管径对称，内中膜不增厚,血流速度正常。双侧颈总动脉分叉处管径对称，内中膜不增厚，...",颈动脉彩超未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,0007273e4495aa6bba7c0767a9b28b36,NaN,NaN,NaN,NaN,NaN
7788,None,双侧乳房形态轮廓正常，层次尚清，腺体稍增厚，局部回声增强，分布不均，局部呈结节样改变。CDF...,双侧乳腺小叶增生,None,None,None,None,None,None,None,...,None,None,None,None,000f4214ad777bdea3d9be0fe06e68ff,NaN,NaN,NaN,NaN,NaN
3865,None,None,膀胱、双侧输尿管未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,00138e755c552ba7cf328765cceb3245,NaN,NaN,NaN,NaN,NaN
6568,None,1.各房室内径正常，主动脉、肺动脉内径正常；2.各瓣膜形态结构及启闭未见异常；3.室间隔、左...,心脏彩超未发现明显异常,None,None,None,None,None,None,None,...,None,None,None,None,001fd1aa107b11e6ef3f62ce6bae3a41,NaN,NaN,NaN,NaN,NaN


In [62]:
test.shape

(9550, 2801)

In [68]:
# 真正的train数据：
test_vid = list(test['vid'].values)
train = train[ ~train['vid'].isin(test_vid)]

In [69]:
train.shape

(47748, 2801)

In [71]:
train.to_csv('./train.csv')
test.to_csv('./test.csv')

In [72]:
## 数据探索
train.describe()

,血清高密度脂蛋白,血清低密度脂蛋白
count,38199.000000,38199.000000
mean,1.406683,2.769719
std,0.341184,0.852217
min,0.150000,-1.220000
25%,1.160000,2.180000
50%,1.350000,2.690000
75%,1.600000,3.260000
max,4.780000,11.540000


In [73]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47748 entries, 11753 to 32778
Columns: 2801 entries, 004997 to 血清低密度脂蛋白
dtypes: float64(2), object(2799)
memory usage: 1020.7+ MB


In [74]:
print (train.loc[1])


004997                                                   None
0101        双侧颈总动脉管径对称，内中膜不增厚,血流速度正常。双侧颈总动脉分叉处管径对称，内中膜不增厚，...
0102                                             颈动脉彩超未发现明显异常
0104                                                     None
0105                                                     None
0106                                                     None
0107                                                     None
0108                                                     None
0109                                                     None
0111                                                     None
0112                                                     None
0113                   肝脏形态较饱满，轮廓规整，肝内回声呈点状密集弥漫性增强，肝内管道结构欠清晰。
0114                           胆囊形态大小正常，壁光滑，内未见明显异常光团。胆总管不扩张。
0115                      大小、形态正常、实质回声均匀，胰管无扩张。CDFI：未见异常血流信号。
0116                                      脾脏形态正常，包膜光整，实质回声均匀。
0117                           左肾形态大小正常，包膜光整，实质回声均匀。集合系统未见分离。
0118    

In [75]:
print (train.dtypes)

004997       object
0101         object
0102         object
0104         object
0105         object
0106         object
0107         object
0108         object
0109         object
0111         object
0112         object
0113         object
0114         object
0115         object
0116         object
0117         object
0118         object
0119         object
0120         object
0121         object
0122         object
0123         object
0124         object
0125         object
0126         object
0127         object
0130         object
0131         object
019001       object
019002       object
             ...   
W49039       object
W49040       object
W49042       object
W49045       object
W59001       object
W69001       object
W69003       object
W99001       object
X19001       object
X19002       object
X19003       object
X19004       object
X19011       object
X19022       object
Y19001       object
Y19004       object
Y29001       object
Y29002       object
Y29003       object


In [81]:
# 对类别型特征，观察其取值范围及直方图
categorical_features = train.select_dtypes(include = ["object"]).columns
for col in categorical_features:
    print ('\n%s属性的不同取值和出现的次数'%col)
    print (train[col].value_counts())


004997属性的不同取值和出现的次数
10.24    60
10.26    50
11.24    47
18.45    40
10.63    38
18.52    36
10.02    32
20.12    27
11.26    24
16.32    20
17.48    20
19.63    19
16.45    18
19.03    17
18.56    16
20.85    16
17.49    14
11.02    14
16.96    13
10.23    13
18.66    12
16.53    12
10.30    12
18.82    11
18.96    11
20.76    11
20.94    11
15.64    11
17.27    11
18.36    11
         ..
11.44     1
23.31     1
19.85     1
8.00      1
19.51     1
26.60     1
16.38     1
23.14     1
13.49     1
11.88     1
9.70      1
6.38      1
13.52     1
23.91     1
12.85     1
21.52     1
14.50     1
24.12     1
13.14     1
17.64     1
6.92      1
16.19     1
12.70     1
23.27     1
12.55     1
13.63     1
12.46     1
6.30      1
14.22     1
7.20      1
Name: 004997, Length: 1511, dtype: int64

0101属性的不同取值和出现的次数
双侧甲状腺大小形态正常，包膜光整，实质回声均匀，未见明显异常回声。CDFI：血流显示未见异常。                                                                                                                                            

脾脏大小、形态正常，包膜光整，回声均匀。                                          19054
脾脏大小、形态正常，包膜光整，内光点均匀。                                         11600
脾脏大小测值正常，回声均匀，脾静脉测值正常。                                         7062
脾脏大小、形态正常，包膜光整，回声均匀，脾门静脉未见明显扩张。CDFI：脾内未见异常血流信号。                1793
脾脏大小、形态正常，包膜光整，内部回声均匀。                                         1534
脾脏：形态正常，肋间不厚，回声均匀，脾静脉不宽，CDFI血流分布正常。                            1495
脾脏形态正常，包膜光整，实质回声均匀。                                             825
脾脏轮廓清，大小形态正常,实质回声均匀，脾静脉内径正常。                                    539
脾脏大小形态尚正常,包膜光滑,回声均匀,其内未见明显异常回声。                                 398
脾脏形态大小正常，表面光滑，脾实质回声均匀，未见异常回声，脾静脉主干内经无扩张。CDFI：未见明显异常血流频谱。        225
脾脏大小、形态正常，包膜光整，实质回声均匀。                                          224
形态正常，肋间不厚，回声均匀，脾静脉不宽，CDFI血流分布正常。                                224
脾脏大小、形态未见明显异常，包膜平整，回声均匀。                                        202
脾脏大小、形态正常，包膜光整，内光点均匀。CDFI：其内未见明显异常血流信号                          153
脾脏大小、形态正常，包膜光整，内光点均匀。CDFI：未见明显异常血流信号            

Name: 019010, dtype: int64

019012属性的不同取值和出现的次数
0.01    3
Name: 019012, dtype: int64

019013属性的不同取值和出现的次数
阴性    1
Name: 019013, dtype: int64

019014属性的不同取值和出现的次数
-     4
阴性    2
Name: 019014, dtype: int64

019015属性的不同取值和出现的次数
<0.5     24
0.8       7
2.40      6
1.5       5
1.6       4
0.3       4
0.40      3
1.0       3
2.00      3
1.1       3
2.50      3
0.4       3
0.00      3
2.80      2
1.8       2
0.58      2
0.6       2
4.00      2
0.1       2
2.30      2
0.30      2
2.3       2
3.2       2
0.68      2
0.9       2
2.65      2
0.64      2
0.29      2
2.7       1
1.96      1
         ..
0.33      1
4.55      1
0.93      1
1.00      1
0.83      1
0.84      1
0.88      1
2.22      1
2.98      1
29.86     1
4.65      1
0.41      1
1.27      1
6.1       1
0.60      1
2.10      1
0.13      1
6.68      1
2.06      1
2.61      1
2.20      1
1.76      1
1.43      1
34.5      1
0.28      1
0.39      1
0.59      1
11.4      1
1.91      1
2.68      1
Name: 019015, Length: 132, dtype: int64

0

0.21     15
0.22     14
0.19     12
0.20     10
0.17      7
0.23      6
0.18      5
0.16      5
0.15      4
0.14      3
0.25      3
0.27      3
0.26      2
0.31      2
0.24      2
0.12      2
0.13      1
0.30      1
0.11      1
0.09      1
0.169     1
0.29      1
Name: 019054, dtype: int64

019055属性的不同取值和出现的次数
9.87     3
10.02    2
15.43    1
30.34    1
13.50    1
10.54    1
20.65    1
14.06    1
22.71    1
12.53    1
13.70    1
11.69    1
5.86     1
8.73     1
21.72    1
16.07    1
13.90    1
10.26    1
9.07     1
17.05    1
20.09    1
10.13    1
12.07    1
12.12    1
34.75    1
10.79    1
12.89    1
29.69    1
19.52    1
13.85    1
        ..
9.50     1
12.03    1
16.32    1
11.26    1
9.30     1
16.70    1
19.91    1
8.22     1
8.60     1
11.76    1
10.03    1
10.29    1
9.83     1
9.55     1
5.58     1
9.98     1
14.26    1
7.92     1
6.69     1
15.79    1
17.60    1
13.61    1
17.84    1
9.28     1
12.80    1
15.89    1
10.84    1
9.63     1
17.22    1
15.90    1
Name: 019055, Len

正常                                         9661
未见异常                                       8120
慢性咽炎                                       3341
咽部粘膜充血                                      546
充血                                          476
咽部滤泡组织增生                                    316
淋巴滤泡散在                                      296
咽部滤泡组织增生, 慢性咽炎                              170
未见明显异常                                      164
充血, 慢性咽炎                                    162
淋巴滤泡散在, 慢性咽炎                                149
咽部滤泡组织增生, 充血, 慢性咽炎                          135
咽炎（单纯性咽炎）                                   120
慢性咽炎, 咽部粘膜充血                                107
慢性咽炎, 充血                                    103
充血, 咽部滤泡组织增生                                 81
充血, 淋巴滤泡散在                                   67
慢性咽炎, 淋巴滤泡散在                                 67
咽部滤泡组织增生, 充血                                 66
充血, 干燥, 慢性咽炎                                 64
咽腔狭小                                    

...

Name: D59059, dtype: int64

D59060属性的不同取值和出现的次数
阴性(-)    6
Name: D59060, dtype: int64

D59061属性的不同取值和出现的次数
阴性(-)    6
Name: D59061, dtype: int64

D59062属性的不同取值和出现的次数
.    1
Name: D59062, dtype: int64

D59066属性的不同取值和出现的次数
<3.4    7
Name: D59066, dtype: int64

D59067属性的不同取值和出现的次数
12.6124    1
Name: D59067, dtype: int64

D79001属性的不同取值和出现的次数
阴性    13
+      1
Name: D79001, dtype: int64

D79003属性的不同取值和出现的次数
阴性    1
Name: D79003, dtype: int64

D79004属性的不同取值和出现的次数
阴性    1
Name: D79004, dtype: int64

D79005属性的不同取值和出现的次数
阴性    1
Name: D79005, dtype: int64

D79006属性的不同取值和出现的次数
阴性    1
Name: D79006, dtype: int64

D79007属性的不同取值和出现的次数
阴性    1
Name: D79007, dtype: int64

D79008属性的不同取值和出现的次数
阴性    1
Name: D79008, dtype: int64

D79009属性的不同取值和出现的次数
阴性    1
Name: D79009, dtype: int64

D79010属性的不同取值和出现的次数
阴性    1
Name: D79010, dtype: int64

D79011属性的不同取值和出现的次数
阴性    1
Name: D79011, dtype: int64

D79012属性的不同取值和出现的次数
阴性    1
Name: D79012, dtype: int64

D79013属性的不同取值和出现的次数
阴性    1
Name: D79013, dtype: int64

0.32    1
0.16    1
2.23    1
Name: F69033, dtype: int64

F69034属性的不同取值和出现的次数
441.30    1
184.39    1
157.90    1
65.29     1
245.82    1
278.90    1
36.97     1
297.23    1
Name: F69034, dtype: int64

F69035属性的不同取值和出现的次数
0.21    2
0.24    1
3.73    1
2.52    1
0.26    1
0.72    1
2.93    1
Name: F69035, dtype: int64

F69053属性的不同取值和出现的次数
阴性    21
-      5
Name: F69053, dtype: int64

F69055属性的不同取值和出现的次数
详见报告单    2
Name: F69055, dtype: int64

F79004属性的不同取值和出现的次数
57.01    1
68.79    1
64.94    1
69.31    1
60.66    1
57.20    1
81.07    1
68.13    1
71.29    1
Name: F79004, dtype: int64

F99001属性的不同取值和出现的次数
11.8    1
Name: F99001, dtype: int64

G49050属性的不同取值和出现的次数
1.67    3
1.57    3
1.88    3
2.00    2
1.83    2
2.03    2
1.82    2
1.84    2
1.75    2
2.25    2
1.53    2
1.71    2
1.52    2
1.70    2
1.76    2
1.47    1
1.41    1
1.59    1
2.13    1
2.04    1
1.96    1
1.90    1
1.73    1
1.85    1
1.74    1
2.11    1
1.95    1
1.86    1
2.07    1
1.98    1
1.69    1
1.58    1
1.97    1


-    2
Name: I49011, dtype: int64

I49012属性的不同取值和出现的次数
-                   212
-        0mmol/L    115
0                   111
0.6                  58
０                     9
０.６                   5
+-                    4
+1     1.4mmol/L      4
+++                   2
+                     2
+3     5.6mmol/L      2
0.2                   1
1.4                   1
２.８                   1
2.8                   1
Name: I49012, dtype: int64

I49016属性的不同取值和出现的次数
4.4    1
Name: I49016, dtype: int64

I49018属性的不同取值和出现的次数
-         7
1：10阳性    1
Name: I49018, dtype: int64

I59002属性的不同取值和出现的次数
127.00    1
116.80    1
-         1
101.20    1
Name: I59002, dtype: int64

I59005属性的不同取值和出现的次数
阴性    10
Name: I59005, dtype: int64

I59006属性的不同取值和出现的次数
阴性    10
Name: I59006, dtype: int64

I69003属性的不同取值和出现的次数
0.29    19
0.21    16
0.24    16
0.23    15
0.25    15
0.28    14
0.35    14
0.31    14
0.32    13
0.34    13
0.33    13
0.22    12
0.30    11
0.36    10
0.26    10
0.18     9
0.40     9
0.17     8


Name: L19007, dtype: int64

L19008属性的不同取值和出现的次数
1.33     8
1.41     7
1.39     7
1.28     5
1.26     5
1.51     5
1.52     5
1.22     4
1.24     4
1.35     4
1.36     4
1.25     4
1.27     4
1.99     3
1.29     3
1.68     3
1.62     3
1.84     3
2.62     3
1.14     3
1.56     3
1.54     3
1.05     3
1.47     2
1.82     2
1.15     2
3.91     2
1.08     2
1.60     2
4.62     2
        ..
4.65     1
1.01     1
34.66    1
1.48     1
1.31     1
1.79     1
8.34     1
10.26    1
11.36    1
1.74     1
1.92     1
3.16     1
2.07     1
1.93     1
1.58     1
2.44     1
8.40     1
8.39     1
3.20     1
10.81    1
6.96     1
1.69     1
2.30     1
4.81     1
1.00     1
14.98    1
3.15     1
9.45     1
6.08     1
7.20     1
Name: L19008, Length: 169, dtype: int64

L29001属性的不同取值和出现的次数
Series([], Name: L29001, dtype: int64)

L49005属性的不同取值和出现的次数
65.5    1
98.7    1
Name: L49005, dtype: int64

L49006属性的不同取值和出现的次数
30.8    1
32.6    1
Name: L49006, dtype: int64

L49007属性的不同取值和出现的次数
46.6    1
73.7    1
Name

Name: T99020, dtype: int64

T99021属性的不同取值和出现的次数
-    8
Name: T99021, dtype: int64

T99022属性的不同取值和出现的次数
-    8
Name: T99022, dtype: int64

T99023属性的不同取值和出现的次数
-    8
Name: T99023, dtype: int64

T99024属性的不同取值和出现的次数
-    8
Name: T99024, dtype: int64

T99025属性的不同取值和出现的次数
-    7
Name: T99025, dtype: int64

T99026属性的不同取值和出现的次数
-    7
Name: T99026, dtype: int64

T99027属性的不同取值和出现的次数
-    7
Name: T99027, dtype: int64

T99028属性的不同取值和出现的次数
-    8
Name: T99028, dtype: int64

U99001属性的不同取值和出现的次数
1+     6
3+     6
2+     4
0-4    2
4+     1
Name: U99001, dtype: int64

U99003属性的不同取值和出现的次数
1+     7
0-2    5
3+     4
0-3    2
2+     1
Name: U99003, dtype: int64

U99004属性的不同取值和出现的次数
未见    15
+      4
Name: U99004, dtype: int64

U99005属性的不同取值和出现的次数
未见    19
Name: U99005, dtype: int64

U99006属性的不同取值和出现的次数
未见    64
Name: U99006, dtype: int64

U99007属性的不同取值和出现的次数
未见    63
+      1
Name: U99007, dtype: int64

U99008属性的不同取值和出现的次数
未见    64
Name: U99008, dtype: int64

U99009属性的不同取值和出现的次数
0      63
4.6     1
Nam